# Assignment: Linear Models
## Do two questions in total: "Q1+Q2" or "Q1+Q3"
### `! git clone https://github.com/ds3001f25/linear_models_assignment.git`

**Q1.** Let's explore multiple linear regression in a two-variable case, to build more intuition about what is happening.

Suppose the model is
$$
\hat{y}_i = b_0 + b_1 z_{i1} + b_2 z_{i2}
$$
Assume that $z_{ij}$ is centered or de-meaned, so that $z_{ij} = x_{ij} - m_j$ where $m_j$ is the mean of variable $j$ and $x_{ij}$ is the original value of variable $j$ for observation $i$. Notice that this implies
$$
\dfrac{1}{N} \sum_{i=1}^N z_{ij} = 0
$$
which will simplify your calculations below substantially!

1. Write down the SSE for this model.
2. Take partial derivatives with respect to $b_0$, $b_1$, and $b_2$.
3. Verify that the average error is zero and $e \cdot z =0$ at the optimum, just as in the single linear regression case.
4. Show that the optimal intercept is $b_0^* = \bar{y}$. Eliminate $b_0^*$ from the remaining equations, and focus on $b_1$ and $b_2$.
5. Write your results as a matrix equation in the form "$Ab=C$". These are called the **normal equations**.
6. Divide both sides by $N$ and substitute $z_{ij} = x_{ij} - m_j$ back into your normal equations for $x_{ij}$. What is the matrix $A$? What is the vector $C$? Explain the intuition of your discovery.

**Q2.** This question is a case study for linear models. The data are about car prices. In particular, they include:

  - `Price`, `Color`, `Seating_Capacity`
  - `Body_Type`: crossover, hatchback, muv, sedan, suv
  - `Make`, `Make_Year`: The brand of car and year produced
  - `Mileage_Run`: The number of miles on the odometer
  - `Fuel_Type`: Diesel or gasoline/petrol
  - `Transmission`, `Transmission_Type`:  speeds and automatic/manual

  1. Load `cars_hw.csv`. These data were really dirty, and I've already cleaned them a significant amount in terms of missing values and other issues, but some issues remain (e.g. outliers, badly scaled variables that require a log or arcsinh transformation). Clean the data however you think is most appropriate.
  2. Summarize the `Price` variable and create a kernel density plot. Use `.groupby()` and `.describe()` to summarize prices by brand (`Make`). Make a grouped kernel density plot by `Make`. Which car brands are the most expensive? What do prices look like in general?
  3. Split the data into an 80% training set and a 20% testing set.
  4. Make a model where you regress price on the numeric variables alone; what is the $R^2$ and `RMSE` on the training set and test set? Make a second model where, for the categorical variables, you regress price on a model comprised of one-hot encoded regressors/features alone (you can use `pd.get_dummies()`; be careful of the dummy variable trap); what is the $R^2$ and `RMSE` on the test set? Which model performs better on the test set? Make a third model that combines all the regressors from the previous two; what is the $R^2$ and `RMSE` on the test set? Does the joint model perform better or worse, and by home much?
  5. Use the `PolynomialFeatures` function from `sklearn` to expand the set of numerical variables you're using in the regression. As you increase the degree of the expansion, how do the $R^2$ and `RMSE` change? At what point does $R^2$ go negative on the test set? For your best model with expanded features, what is the $R^2$ and `RMSE`? How does it compare to your best model from part 4?
  6. For your best model so far, determine the predicted values for the test data and plot them against the true values. Do the predicted values and true values roughly line up along the diagonal, or not? Compute the residuals/errors for the test data and create a kernel density plot. Do the residuals look roughly bell-shaped around zero? Evaluate the strengths and weaknesses of your model.

**Q3.** This question refers to the `heart_hw.csv` data. It contains three variables:

  - `y`: Whether the individual survived for three years, coded 0 for death and 1 for survival
  - `age`: Patient's age
  - `transplant`: `control` for not receiving a transplant and `treatment` for receiving a transplant

Since a heart transplant is a dangerous operation and even people who successfully get heart transplants might suffer later complications, we want to look at whether a group of transplant recipients tends to survive longer than a comparison group who does not get the procedure.

1. Compute (a) the proportion of people who survive in the control group who do not receive a transplant, and (b) the difference between the proportion of people who survive in the treatment group and the proportion of people who survive in the control group. In a randomized controlled trial, this is called the **average treatment effect**.
2. Regress `y` on `transplant` using a linear model with a constant. How does the constant/intercept of the regression and the coefficient on transplant compare to your answers from part 1? Explain the relationship clearly.
3. We'd like to include `age` in the regression, since it's reasonable to expect that older patients are less likely to survive an extensive surgery like a heart transplant. Regress `y` on a constant, transplant, and age. How does the intercept change?
4. Build a more flexible model that allows for non-linear age effects and interactions between age and treatment. Use a train-test split to validate your model. Estimate your best model, predict the survival probability by age, and plot your results conditional on receiving a transplant and not. Describe what you see.
5. Imagine someone suggests using these kinds of models to select who receives organ transplants; perhaps the CDC or NIH starts using a scoring algorithm to decide who is contacted about a potential organ. What are your concerns about how it is built and how it is deployed?

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
df = pd.read_csv('/heart_hw.csv')

Q3.1

In [8]:
control_group = df[df['transplant'] == 'control']
treatment_group = df[df['transplant'] == 'treatment']

prop_control = control_group['y'].mean()
print("Proportion of patients who do not receive a transplant in the control group and survive:", prop_control)

prop_treatment = treatment_group['y'].mean()

diff_prop = prop_treatment - prop_control
print("Difference in survival proportions:", diff_prop)

Proportion of patients who do not receive a transplant in the control group and survive: 0.11764705882352941
Difference in survival proportions: 0.23017902813299232


Q3.2

In [9]:
from sklearn.linear_model import LinearRegression

y = df['y']
vars = ['transplant']

df['transplant_num'] = (df['transplant'] == 'treatment').astype(int)

vars = ['transplant_num']

X = df.loc[:, vars]

reg = LinearRegression().fit(X, y)

# Print results
print("Intercept:", reg.intercept_)
print("Coefficient on transplant:", reg.coef_)
print("R-squared:", reg.score(X, y))


Intercept: 0.1176470588235293
Coefficient on transplant: [0.23017903]
R-squared: 0.059188892948483174


The intercept represents the mean survival probability in the control group. Its value, ~11.8%, exactly matches the value I got in part 1.
The coefficient on transplant is the difference between treatment and control survival. In this part and part 1, I got a value of ~0.23.

Q3.3

In [12]:
df['transplant'] = (df['transplant'] == 'treatment').astype(int)

vars = ['transplant', 'age']
X = df.loc[:, vars]

reg2 = LinearRegression().fit(X, y)

print("Intercept:", reg2.intercept_)
print("Coefficients:", reg2.coef_)
print("R-squared:", reg2.score(X, y))


Intercept: 0.7019569721740947
Coefficients: [ 0.26470169 -0.01360722]
R-squared: 0.1467806594163431


The intercept increased from 0.118 to 0.702 because this model accounts for age, meaning it interprets the baseline differently.

Q3.4

In [16]:
from sklearn.model_selection import train_test_split

df['age_sq'] = df['age'] ** 2
df['age_transplant'] = df['age'] * df['transplant']

y = df['y']
vars = ['transplant', 'age', 'age_sq', 'age_transplant']
X = df.loc[:, vars]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


reg3 = LinearRegression().fit(X_train, y_train)

print("Train R²:", reg3.score(X_train, y_train))
print("Test R²:", reg3.score(X_test, y_test))


Train R²: 0.1582536228841992
Test R²: 0.212732813392986


In [17]:
age_grid = np.linspace(df['age'].min(), df['age'].max(), 100)

X_control = pd.DataFrame({
    'transplant': 0,
    'age': age_grid,
    'age_sq': age_grid ** 2,
    'age_transplant': 0 * age_grid
})
pred_control = reg3.predict(X_control)

X_treatment = pd.DataFrame({
    'transplant': 1,
    'age': age_grid,
    'age_sq': age_grid ** 2,
    'age_transplant': 1 * age_grid
})
pred_treatment = reg3.predict(X_treatment)

plt.figure(figsize=(8, 5))
plt.plot(age_grid, pred_control, label='Control (No Transplant)')
plt.plot(age_grid, pred_treatment, label='Treatment (Transplant)')
plt.xlabel('Age')
plt.ylabel('Predicted Survival Probability')
plt.title('Predicted Survival by Age and Transplant Status')
plt.legend()
plt.grid(True)
plt.show()

The predicted survival probability for transplant recipients starts high for younger patients and decreases steadily with age, reaching 0% around age 62. The control group starts lower for young patients (~30%) and declines to 0% by age 58. The gap between treatment and control is largest for younger patients and narrows with age, showing that transplants are predicted to be most beneficial for younger patients.

Q3.5

Using a predictive model to choose who receives transplants raises several concerns. The model captures correlations in data but doesn't guarantee causal effects, so relying on it could misrepresent who actually benefits. Models may also reflect biases in the training data, leading to unfair treatment of certain groups. Additionally, small sample sizes or missing patient types can make predictions unreliable. Models should not replace medical decision making but can be used to support it.